In [8]:
import random
import numpy as np
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

# Create a synthetic dataset
data = [
    {"tokens": ["Le", "patient", "a", "de", "la", "fièvre", "et", "des", "maux", "de", "tête", "."],
     "ner_tags": [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Les", "plaintes", "incluent", "des", "difficultés", "respiratoires", "et", "des", "douleurs", "thoraciques", "."],
     "ner_tags": [0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]},
    {"tokens": ["Les", "symptômes", "signalés", "sont", "des", "maux", "de", "tête", "et", "des", "nausées", "."],
     "ner_tags": [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "décrit", "des", "douleurs", "musculaires", "et", "une", "faiblesse", "."],
     "ner_tags": [0, 0, 0, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["Les", "symptômes", "rapportés", "incluent", "un", "mal", "de", "gorge", "et", "un", "nez", "qui", "coule", "."],
     "ner_tags": [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Les", "symptômes", "observés", "sont", "des", "douleurs", "abdominales", "et", "des", "vomissements", "."],
     "ner_tags": [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "se", "plaint", "de", "douleurs", "articulaires", "et", "de", "gonflements", "."],
     "ner_tags": [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]},
    {"tokens": ["Les", "symptômes", "incluent", "des", "vertiges", "et", "une", "perte", "d'appétit", "."],
     "ner_tags": [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]},
    {"tokens": ["Le", "patient", "rapporte", "des", "douleurs", "dorsales", "et", "des", "difficultés", "à", "marcher", "."],
     "ner_tags": [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0]},
    {"tokens": ["Les", "symptômes", "consistent", "en", "des", "éruptions", "cutanées", "et", "des", "démangeaisons", "."],
     "ner_tags": [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]}
]

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({
    "tokens": [item["tokens"] for item in data],
    "ner_tags": [item["ner_tags"] for item in data]
})

# Define label list and other necessary variables
label_list = ["O", "B-SYMPTOM", "I-SYMPTOM"]
num_labels = len(label_list)

# Load tokenizer and model
model_name = "camembert-base"
tokenizer = CamembertTokenizerFast.from_pretrained(model_name)
model = CamembertForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, padding='max_length', max_length=128, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Split into train and validation sets
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Load the evaluation metric
metric = load_metric("seqeval")

# Define a compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

C:\Users\saeed\anaconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 1.1372319459915161, 'eval_precision': 0.3125, 'eval_recall': 0.8333333333333334, 'eval_f1': 0.45454545454545453, 'eval_accuracy': 0.3181818181818182, 'eval_runtime': 0.999, 'eval_samples_per_second': 2.002, 'eval_steps_per_second': 1.001, 'epoch': 3.0}
